In [1]:
import os
import math
import csv
import pickle
import statistics
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import sample,choices,choice,uniform, randint,shuffle
from DSSCircuit_Interface import *

In [2]:
#Read the .gml graph and load it into notebook, get the node list
FolderName=os.path.dirname(os.path.realpath("__file__"))
G_original = nx.read_gml(""+ FolderName+ r"\34busEx.gml")
#nodes=list(G_original.nodes())
#edges=list(G_original.edges())
source_node = 'sourcebus'

In [3]:
DSSfile=r""+ FolderName+ "\ieee34Mod2.dss" 

In [4]:
list(G_original.edges(data=True))

[('sourcebus',
  '800',
  {'label': 'subxf',
   'device': 'Transformer',
   'resistance': 0.012400199999999998,
   'reactance': 0.024800399999999997,
   'phases': 3,
   'maxcap': 313.7773202117531,
   'cap': 16.50637143694331,
   'residue': 297.27094877480977}),
 ('800',
  '802',
  {'label': 'l1',
   'device': 'Line',
   'resistance': 0.5473215904400001,
   'reactance': 0.40716439247999997,
   'phases': 3,
   'maxcap': 600.0,
   'cap': 45.77337931904375,
   'residue': 554.2266206809562}),
 ('802',
  'mid806',
  {'label': 'l2a',
   'device': 'Line',
   'resistance': 0.1835012309033333,
   'reactance': 0.13651054244,
   'phases': 3,
   'maxcap': 600.0,
   'cap': 45.779600967482736,
   'residue': 554.2203990325172}),
 ('mid806',
  '806',
  {'label': 'l2b',
   'device': 'Line',
   'resistance': 0.1835012309033333,
   'reactance': 0.13651054244,
   'phases': 3,
   'maxcap': 600.0,
   'cap': 44.51783080175414,
   'residue': 555.4821691982459}),
 ('806',
  '808',
  {'label': 'l3',
   'device'

In [5]:
d_obj= DSS(DSSfile) #create a circuit object
d_obj.compile_ckt_dss() #compiling the circuit
d_obj.dssText.command = "Set Maxiterations=200" 
d_obj.dssText.command = "Set maxcontroliter=200"
d_obj.AllowForms=False 
d_obj.dssText.command = "Set controlmode=OFF"

In [6]:
#Loading Original Loadshape File
LoadShape = []
with open('Loadshape_multload.csv') as csvfile:
    data = list(csv.reader(csvfile))

for x in range(len(data)):
    value = float(data[x][1])
    LoadShape.append(value)

In [7]:
#Creating Array of Loadshapes
Loadshapes = []
Loadshapes.append(LoadShape)
std = statistics.stdev(LoadShape)
mean = statistics.mean(LoadShape)
length = len(LoadShape)
Nsc_N = 200 
for i in range(Nsc_N):
    noise = np.random.normal(mean,std,length)
    Loadshapes.append(list(noise))

In [8]:
#Generating Normal Voltage signals
buses=list(G_original.nodes())
edge_dict=list(G_original.edges(data=True)) 
Prelim_Dataset = []
d_obj.dssText.command='redirect LoadShape.dss'
d_obj.dssCircuit.Loadshapes.First
for i in range(len(Loadshapes)):
    d_obj.dssText.command = 'New LoadShape.Lshape'+str(i) + ' npts=24 interval=1 mult='+str(Loadshapes[i])
    d_obj.dssCircuit.Loadshapes.Name = 'Lshape'+str(i)
    d_obj.dssText.command ='Batchedit Load..* Daily=Lshape'+str(i)
    d_obj.dssText.command = "Set mode=Daily"
    d_obj.dssText.command = "Set stepsize=1h"
    d_obj.dssText.command = "Set number=1"
    d_obj.dssCircuit.Loadshapes.Next
    Normal_Data = []
    TimeSeries_Data = []
    dsstime = 0
    for b in buses:
        d_obj.dssCircuit.SetActiveBus(b) #Setting Active Bus
        while dsstime<24: 
            d_obj.dssText.Command= 'Solve'
            dsstime = d_obj.dssSolution.dblHour
            V=np.array(d_obj.dssBus.puVmagAngle)
            ctidx = 2 * np.array(range(0, min(int(V.size/ 2), 3)))
            TimeSeries_Data.append({'Time Step':dsstime, 'Voltage Magnitude':V[ctidx]})
        Normal_Data.append({'Bus Name':b,'Time Series Data':TimeSeries_Data})
    Scenario_Info={'Scaling Factor':'Nil', 'Anomaly_Duration':'Nil', 'targeted_bus': 'Nil', 'Scenario_Info': Normal_Data}
    Prelim_Dataset.append(Scenario_Info)

In [10]:
len(Prelim_Dataset)

201

In [9]:
#Injecting Anomalies
Nsc_A = 299 #Number of Anomalous Scenarios
Ttot = 24 
for a in range (Nsc_A):
    Anomaly_Data = []
    selected_bus = choice(buses) #random sampling of buses with replacement
    lamda = uniform(-1.0,0.5) #Anomaly Scaling Factor
    time_range = randint(0,5) #Duration of Anomaly in hours
    anom_start = randint(0,19) #Anomaly Start Time
    for j in range(len(buses)):
        NewTimeSeries_Data = []
        for m in range(Ttot):
            if Prelim_Dataset[0]['Scenario_Info'][j]['Bus Name'] == selected_bus and Prelim_Dataset[0]['Scenario_Info'][j]['Time Series Data'][m]['Time Step'] > anom_start and Prelim_Dataset[0]['Scenario_Info'][j]['Time Series Data'][m]['Time Step'] <= anom_start+time_range:
                New_VM = (1+lamda)*Prelim_Dataset[0]['Scenario_Info'][j]['Time Series Data'][m]['Voltage Magnitude']
            else:
                New_VM = Prelim_Dataset[0]['Scenario_Info'][j]['Time Series Data'][m]['Voltage Magnitude']      
            NewTimeSeries_Data.append({'Time Step':Prelim_Dataset[0]['Scenario_Info'][j]['Time Series Data'][m]['Time Step'], 'Voltage Magnitude':New_VM})
        Anomaly_Data.append({'Bus Name':Prelim_Dataset[0]['Scenario_Info'][j]['Bus Name'],'Time Series Data': NewTimeSeries_Data})          
    Scenario_Info = {'Scaling Factor':lamda, 'Anomaly_Duration':time_range, 'targeted_bus': selected_bus, 'Scenario_Info': Anomaly_Data}
    Prelim_Dataset.append(Scenario_Info)
 

In [10]:
#Restructuring Dataset into multi-dimensional array
Dataset = []
dim_x = len(NewTimeSeries_Data)
dim_y = len(buses)

for d in range (len(Prelim_Dataset)):
    array_3d = np.zeros((dim_x,dim_y,3))
    array_2d = np.zeros((dim_y,3))
    for t in range(len(NewTimeSeries_Data)):
        for s in range(len(buses)): 
            array_2d[s,:] = Prelim_Dataset[d]['Scenario_Info'][s]['Time Series Data'][t]['Voltage Magnitude']
        array_3d[t,:,:]=array_2d
        if d>Nsc_N+1 and np.any(array_3d<0.95) or np.any(array_3d>1.05):
            Anomaly_Check = 'Yes' 
        else:
            Anomaly_Check = 'No'              
    Scenario_Data = {'Anomaly Check':Anomaly_Check,'targeted_bus':Prelim_Dataset[d]['targeted_bus'],'TimeSeries_Voltage':array_3d}
    Dataset.append(Scenario_Data)

In [12]:
Dataset[0]

{'Anomaly Check': 'No',
 'targeted_bus': 'Nil',
 'TimeSeries_Voltage': array([[[0.99999144, 0.99999391, 0.99999384],
         [0.99999144, 0.99999391, 0.99999384],
         [0.99999144, 0.99999391, 0.99999384],
         ...,
         [0.99999144, 0.99999391, 0.99999384],
         [0.99999144, 0.99999391, 0.99999384],
         [0.99999144, 0.99999391, 0.99999384]],
 
        [[0.99999215, 0.9999944 , 0.99999436],
         [0.99999215, 0.9999944 , 0.99999436],
         [0.99999215, 0.9999944 , 0.99999436],
         ...,
         [0.99999215, 0.9999944 , 0.99999436],
         [0.99999215, 0.9999944 , 0.99999436],
         [0.99999215, 0.9999944 , 0.99999436]],
 
        [[0.99999239, 0.99999457, 0.99999453],
         [0.99999239, 0.99999457, 0.99999453],
         [0.99999239, 0.99999457, 0.99999453],
         ...,
         [0.99999239, 0.99999457, 0.99999453],
         [0.99999239, 0.99999457, 0.99999453],
         [0.99999239, 0.99999457, 0.99999453]],
 
        ...,
 
        [[0.999988

In [13]:
Prelim_Dataset[300]

{'Scaling Factor': -0.8491016986936497,
 'Anomaly_Duration': 1,
 'targeted_bus': '707',
 'Scenario_Info': [{'Bus Name': 'sourcebus',
   'Time Series Data': [{'Time Step': 1.0,
     'Voltage Magnitude': array([0.99999144, 0.99999391, 0.99999384])},
    {'Time Step': 2.0,
     'Voltage Magnitude': array([0.99999215, 0.9999944 , 0.99999436])},
    {'Time Step': 3.0,
     'Voltage Magnitude': array([0.99999239, 0.99999457, 0.99999453])},
    {'Time Step': 4.0,
     'Voltage Magnitude': array([0.99999273, 0.99999481, 0.99999479])},
    {'Time Step': 5.0,
     'Voltage Magnitude': array([0.99999277, 0.99999484, 0.99999482])},
    {'Time Step': 6.0,
     'Voltage Magnitude': array([0.99999247, 0.99999463, 0.9999946 ])},
    {'Time Step': 7.0,
     'Voltage Magnitude': array([0.99999172, 0.9999941 , 0.99999405])},
    {'Time Step': 8.0,
     'Voltage Magnitude': array([0.99999045, 0.99999322, 0.99999311])},
    {'Time Step': 9.0,
     'Voltage Magnitude': array([0.99998923, 0.99999239, 0.99999

In [14]:
#Dataset Balance Check
j = 0
for i in range(len(Dataset)):
    if Dataset[i]['Anomaly Check'] == 'No':
        j = j+1 
j

271

In [15]:
#Shuffling Dataset and Shuffle Check
shuffle(Dataset)
for i in range(len(Dataset)):
    print(Dataset[i]['Anomaly Check'])

No
Yes
No
Yes
No
Yes
No
No
No
No
No
Yes
No
No
No
No
Yes
Yes
No
No
Yes
Yes
Yes
Yes
No
No
No
No
Yes
No
Yes
No
No
No
Yes
No
No
Yes
No
Yes
Yes
No
No
Yes
No
Yes
Yes
Yes
No
Yes
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
No
Yes
No
Yes
Yes
No
No
No
No
Yes
Yes
No
Yes
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
Yes
No
No
Yes
No
No
Yes
Yes
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
No
No
Yes
No
No
No
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
Yes
No
No
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
No
No
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
No
No
Yes
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
No
Yes
No
No
Yes
Yes
No
Yes
Yes
No
Yes
No
Yes
No
Yes
No
No
No
No
No
No
No
No
Yes
Yes
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
No
Yes
No
No
Yes
No
Yes
Yes
No
Yes
No
No
No
No
No
No
No
Yes
No
Yes
No
No
Yes
Yes
No
Yes
No
No
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
Yes
Yes
Yes
No
Yes
No
Y

In [16]:
#Loading .pkl file
a = open('fullanomalydataset.pkl', 'wb')
pickle.dump(Dataset,a)
a.close()